## Select the analysis mode from the dropdown menu below.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from measurement_directory import *

def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'MATLAB'])

interactive(children=(Dropdown(description='analysis_mode', options=('fake analysis', 'fake analysis 2', 'MATL…

## Enter the run from today you wish to analyze.

In [2]:
global watchfile
measurement_dir = measurement_directory()
watchfile = measurement_dir + r'\image_log.csv'

Enter name for this set of runs:test


## Run the cell below to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [3]:
print('loading matlab engine...')
import matlab.engine
eng = matlab.engine.start_matlab()
print('matlab engine loaded')

loading matlab engine...
matlab engine loaded


In [ ]:
import os
import time
import datetime
import shutil
import posixpath
import sys
# from breadboard import BreadboardClient
import warnings
import pandas as pd
from numpy import isnan
import ipywidgets as widgets
from ipywidgets import interact


# analysis_type = 'fake analysis'

# define analysis_function which accepts image filepath and returns dictionary of {analyzed_var_name:analysis_value}
if analysis_type == 'fake analysis':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,5))
        return {'fake analysis 1': randint(0,42), 'fake analysis 2': randint(0,42)}
    analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
elif analysis_type == 'fake analysis 2':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,5))
        return {'fake analysis 3': randint(0,42), 'fake analysis 4': randint(0,42)}
    analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
elif analysis_type == 'MATLAB':
    analyzed_var_names = ['bareNcntAverageMarqueeBoxValues', 
                          'COMX', 'COMY'] #TODO set MATLAB analyzed_var_names properly
    from matlab_wrapper import getMATLABanalysis
#     from random import randint
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
#             print('no previous settings')
#             return {'fakematlab1': randint(0,42), 'fakematlab2': randint(0,42)}, 'foo'
            matlab_dict = getMATLABanalysis(eng, filepath)
        else:
#             print('previous settings: ' + previous_settings)
#             return {'fakematlab1': randint(0,42), 'fakematlab2': randint(0,42)}, 'foo1'
            matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
                                           normBox = previous_settings['normBox'])
        analysis_dict, settings = matlab_dict['analysis'], matlab_dict['settings']
        return analysis_dict, settings

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
# bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

def translate_stringy_list(stringy_list):
    #pd.read_csv imports listboundvariables dtype as a string, we want it in list form
    final_list = []
    stringy_list = stringy_list.split(',')
    for var in stringy_list:
        final_list.append(var.translate({ord(c): None for c in '[ ]\,\''}))
    return final_list

def analysis_to_imagelog(image_log_df, idx, outputlog_filename, previous_settings = None,
                        output_previous_settings = True):
    image_filename = image_log_df.loc[idx, 'filename0']
    abs_image_path = os.path.join(os.path.join(os.getcwd(), measurement_dir), image_filename)
    print(abs_image_path)
    settings = None
    if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
        analysis_dict = analysis_function(image_filename)
    elif analysis_type == 'MATLAB' and previous_settings is None:
        analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
    elif analysis_type == 'MATLAB':
        analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
    if not output_previous_settings:
        settings = None #forces user to select new marquee box for each shot
    image_log_df = pd.read_csv(outputlog_filename) #get updated copy of log to prevent overwriting
    print('{file} analyzing: '.format(file=image_filename))
    for key in analyzed_var_names:
        print(key, analysis_dict[key])
        image_log_df.loc[idx,key] = analysis_dict[key]
    image_log_df.to_csv(outputlog_filename, index = False)
#     print('{file} analyzed: '.format(file=image_filename) + str(analysis_dict))
    return image_log_df, settings

def main():
    # Global settings
    # bc = BreadboardClient(config_path='API_CONFIG.json') # enter your path to the API_config
    refresh_time = 1  # seconds
    global watchfile
    previous_settings = None
#     watchfile = r'07\200729\foo\image_log.csv'
    print("\n\n Watching this file for changes: " + watchfile + "\n\n")

#     length_old = 0

    # Main Loop
    while True:
        if not os.path.exists(watchfile):
            time.sleep(refresh_time)
            continue
        else:
            image_log_df = pd.read_csv(watchfile)

        for var_name in analyzed_var_names:
            if var_name not in image_log_df.columns: #start new analysis type
#             print('{analysis} analysis started'.format(analysis = analysis_type))
                image_log_df, previous_settings = analysis_to_imagelog(image_log_df, 0, watchfile, previous_settings)
                if 'analyzed_variables' not in image_log_df.columns:
                    image_log_df.loc[0,'analyzed_variables'] = str(analyzed_var_names)
                    image_log_df.to_csv(watchfile, index = False)
                else:
                    old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[0,'analyzed_variables']))
                    updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                    image_log_df.loc[0,'analyzed_variables'] = str(updated_analyzed_variables)
                    image_log_df.to_csv(watchfile, index = False)
            else: #queue un-analyzed images
                for idx in reversed(range(len(image_log_df.index))): #start from most recent image first
                    if isnan(image_log_df.loc[idx, var_name]):
                        image_log_df, previous_settings = analysis_to_imagelog(image_log_df, idx, watchfile, previous_settings)
                        if isinstance(image_log_df.loc[idx,'analyzed_variables'],float):
                            image_log_df.loc[idx,'analyzed_variables'] = str(analyzed_var_names)
                            image_log_df.to_csv(watchfile, index = False)
                        else:
                            old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[idx,'analyzed_variables']))
                            updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
                            image_log_df.loc[idx,'analyzed_variables'] = str(updated_analyzed_variables)
                            image_log_df.to_csv(watchfile, index = False)

    # check if a new image has come in
#         if len(image_log_df.index) > length_old:
#             image_filename = image_log_df.loc[len(
#                 image_log_df.index) - 1, 'filename']
#             print('New image: ' + image_filename + '. Analyzing now...')
#             analysis_variable = analysis_function(image_filename)
#             image_log_df.loc[len(image_log_df.index) - 1,
#                              analysis_type] = analysis_variable
#             image_log_df.to_csv('image_log.csv', index = False)
#             print('analysis value ' + str(analysis_variable))

            # # Write to Breadboard
            # try:
            #     resp = bc.post_images(
            #                 image_names = os.path.splitext(output_filename)[0],
            #                 filepath = output_filepath,
            #                 image_times = [dt],
            #                 auto_time = False # Add more information here
            #                 )
            #     if resp.status_code!=200:
            #         print(resp.text)
            # except: pass
            # names_old = names
#             length_old = len(image_log_df.index)

        # Wait before checking again
        time.sleep(refresh_time)

main()



 Watching this file for changes: 08\200805\test\image_log.csv


C:\Users\FermiCam2\Desktop\GitHub\enrico\08\200805\test\test_232.spe
test_232.spe analyzing: 
bareNcntAverageMarqueeBoxValues 14237.598396278563
COMX 151.50631366276474
COMY 187.34559018470696
test_232.spe analyzed: {'bareNcntValues': matlab.double([]), 'bareNcntAverageMarqueeBoxValues': 14237.598396278563, 'fitBimodal': {'parameterNames': ['Amp BEC', 'Amp Therm:BEC ratio', 'Offset', 'BEC ratio width', 'Therm width', 'xBEC', 'xTherm'], 'xparam': matlab.double([[0.2933384223477066,0.6428125570470216,0.0021437355969993847,64.02340066659956,74.50414814962296,148.3750992393326,153.87599418188748]]), 'yparam': matlab.double([[0.2867874807419313,0.5558473299321232,-0.0004718147876813584,71.10284386196332,110.43665995961544,181.94937573150344,191.627581369523]]), 'xci': matlab.double([[[0.2693420925812517,0.518924374031755,-0.0030827796201138118,62.61616061574958,68.31303140696674,147.78205479824632,152.56157310668692],[0.31733

In [9]:
import matlab_wrapper_debug